In [ ]:
from brian2 import *

In [ ]:
start_scope()

In [ ]:
DT=0.1 # time step
defaultclock.dt = DT*ms

In [ ]:
TotTime=1000 #Simulation duration (ms)
duration = TotTime*ms

In [ ]:
FRout_inh=[]
FRout_exc=[]
muve=[]
muvi=[]

In [ ]:
Adapt=[]
Npts=60 #Resolution of the measured TF
Npts2=60
i=0
for rate_exc in linspace(0,60, Npts):
	FRout_inh.append([])
	FRout_exc.append([])
	Adapt.append([])
	muve.append([])	
	muvi.append([])
	for rate_inh in linspace(0,60, Npts2):

In [ ]:
		#(0.04*v**2+5*v+140-u-GsynE*(v-Ee)-GsynI*(v-Ei)-I)/Tn:1
		eqs="""
		dv/dt=(g_iz*(v-E_iz)**2-u-GsynE*(v-Ee)-GsynI*(v-Ei)-I)/Tv:1 (unless refractory)
		du/dt=(a*(b*v-u))/Tu:1
		dGsynI/dt = -GsynI/Tsyn : 1
		dGsynE/dt = -GsynE/Tsyn : 1  
		a:1
		b:1
		c:1
		d:1
		g_iz:1
		E_iz:1
		Tsyn:second
		Tv:second
		Tu:second
		Ee:1
		Ei:1
		I:1
		"""
		#Pvar:1
		#Qi:1
		#Qe:1
		#"""

In [ ]:
		# Population 1 - Fast Spiking

In [ ]:
		G_inh = NeuronGroup(1, eqs, threshold='v > -30.', reset='v = c; u=u+d',refractory='5*ms', method='heun')
		G_inh.v=-50
		G_inh.u=0
		G_inh.a=0.#0.1
		G_inh.b=0.#0.2
		G_inh.c=-55
		G_inh.d=0.0
		G_inh.Tsyn=5.*ms
		G_inh.Tv=1*second
		G_inh.Tu=1*second
		G_inh.Ee=0.0
		G_inh.Ei=-80.
		G_inh.I=0.
		G_inh.g_iz=0.04
		G_inh.E_iz=-60

		
<br>
		G_inh.v=-50<br>
		G_inh.u=0<br>
		G_inh.a=0.#0.1<br>
		G_inh.b=0.#0.2<br>
		G_inh.c=-65<br>
		G_inh.d=0.0<br>
		G_inh.Tsyn=5.*ms<br>
		G_inh.Tv=1*second<br>
		G_inh.Tu=1*second<br>
		G_inh.Ee=0.0<br>
		G_inh.Ei=-80.<br>
		G_inh.I=0.<br>
		G_inh.g_iz=0.04#0.001#0.04<br>
		G_inh.E_iz=-60<br>



In [ ]:
		# Population 2 - Regular Spiking
		G_exc = NeuronGroup(1, eqs, threshold='v > -30.', reset='v = c; u=u+d',refractory='5*ms', method='heun')
		G_exc.v=-50
		G_exc.u=0#25
		G_exc.a=1
		G_exc.b=0.0#0.2
		G_exc.c=-65
		G_exc.d=15.0#8.0
		G_exc.Tsyn=5.*ms
		G_exc.Tv=1*second
		G_exc.Tu=1*second
		G_exc.Ee=0.0
		G_exc.Ei=-80.
		G_exc.I=0.0
		G_exc.g_iz=0.01
		G_exc.E_iz=-65
			
		
		'''
		G_exc.v=-40
		G_exc.u=0.#25
		G_exc.a=0.02
		G_exc.b=0.#0.2
		G_exc.c=-60
		G_exc.d=0.0#8.0
		G_exc.Tsyn=5.*ms
		G_exc.Tv=1*second
		G_exc.Tu=1*second
		G_exc.Ee=0.0
		G_exc.Ei=-80.
		G_exc.I=0.0
		G_exc.g_iz=0.04#0.001#0.04
		G_exc.E_iz=-60
		'''
		# external drive--------------------------------------------------------------------------

In [ ]:
		P_ed_inh = PoissonGroup(100, rates=rate_inh*Hz) #5% of 2000 inh cells
		P_ed_exc = PoissonGroup(400, rates=rate_exc*Hz) #5% of 8000 exc cells

In [ ]:
		# Network-----------------------------------------------------------------------------

In [ ]:
		# connections-----------------------------------------------------------------------------
		#seed(0)
		Qi=5.0#e-6 #*nS
		Qe=1.5#e-6 #*nS

In [ ]:
		prbC=1. #0.05
		 

In [ ]:
		S_edin_in = Synapses(P_ed_inh, G_inh, on_pre='GsynI_post+=Qi')
		S_edin_in.connect(p=prbC)

In [ ]:
		S_edin_ex = Synapses(P_ed_inh, G_exc, on_pre='GsynI_post+=Qi')
		S_edin_ex.connect(p=prbC)

In [ ]:
		S_edex_in = Synapses(P_ed_exc, G_inh, on_pre='GsynE_post+=Qe')
		S_edex_in.connect(p=prbC)

In [ ]:
		S_edex_ex = Synapses(P_ed_exc, G_exc, on_pre='GsynE_post+=Qe')
		S_edex_ex.connect(p=prbC)

In [ ]:
		# Recording tools -------------------------------------------------------------------------------
		M2G2_vi = StateMonitor(G_inh, 'v', record=0)
		M2G2_v = StateMonitor(G_exc, 'v', record=0)
		M3G2 = StateMonitor(G_exc, 'u', record=0)
		FRG_inh = PopulationRateMonitor(G_inh)
		FRG_exc = PopulationRateMonitor(G_exc)

In [ ]:
		# Run simulation -------------------------------------------------------------------------------

In [ ]:
		#print('--##Start simulation##--')
		run(duration)
		#print('--##End simulation##--')

In [ ]:
		#  -------------------------------------------------------------------------------

In [ ]:
		# prepare firing rate
		def bin_array(array, BIN, time_array):
		    N0 = int(BIN/(time_array[1]-time_array[0]))
		    N1 = int((time_array[-1]-time_array[0])/BIN)
		    return array[:N0*N1].reshape((N1,N0)).mean(axis=1)

In [ ]:
		BIN=5
		time_array = arange(int(TotTime/DT))*DT

In [ ]:
		print(test)
		
		LfrG_exc=array(FRG_exc.rate/Hz)
		TimBinned,popRateG_exc=bin_array(time_array, BIN, time_array),bin_array(LfrG_exc, BIN, time_array)

In [ ]:
		LfrG_inh=array(FRG_inh.rate/Hz)
		TimBinned,popRateG_inh=bin_array(time_array, BIN, time_array),bin_array(LfrG_inh, BIN, time_array)
		
		uu=M3G2[0].u
		vv=M2G2_v[0].v
		vvi=M2G2_vi[0].v
		#print(mean(popRateG_inh[150::]))
		#print(mean(popRateG_exc[150::]))
		FRout_inh[i].append(mean(popRateG_inh[int(0.9*len(popRateG_inh))::]))
		FRout_exc[i].append(mean(popRateG_exc[int(0.9*len(popRateG_exc))::]))
		Adapt[i].append(mean(uu[int(0.9*len(uu)):len(uu)]))
		muve[i].append(mean(vv[int(0.9*len(uu)):len(uu)]))
		muvi[i].append(mean(vvi[int(0.9*len(uu)):len(uu)]))
	i=i+1
	
	if i % 1 == 0:
		print(i)

In [ ]:
np.save('ExpTF_Adapt_Nstp-60_Nstp2-60_new-params_noad_0-100_new-mean.npy', Adapt)
np.save('ExpTF_inh_Nstp-60_Nstp2-60_new-params_noad_0-100_new-mean.npy', FRout_inh)
np.save('ExpTF_exc_Nstp-60_Nstp2-60_new-params_noad_0-100_new-mean.npy', FRout_exc)
np.save('ExpTF_muve_Nstp-60_Nstp2-60_new-params_noad_0-100_new-mean.npy', muve)
np.save('ExpTF_muvi_Nstp-60_Nstp2-60_new-params_noad_0-100_new-mean.npy', muvi)

In [ ]:
plt.figure()
plt.imshow(FRout_inh)
plt.colorbar()
plt.figure()
plt.imshow(FRout_exc)
plt.colorbar()
plt.figure()
plt.imshow(Adapt)
plt.colorbar()
plt.show()